https://youtu.be/0ZqeBEa_MWo?si=euGFINmBNnY8qPbn&t=149

In [ ]:
import holoviews as hv
import numpy as np
import pandas as pd
from icecream import ic

In [ ]:
hv.extension("bokeh")

In [ ]:
spool_radius = 5.75
gear_ratio = (59.0 / 17.0) ** 2
STEP_DIVISION = 1
motor_steps_per_revolution = 100 * STEP_DIVISION
spool_circumfrence = spool_radius * 2 * np.pi
ic(spool_circumfrence)
steps_per_mm = motor_steps_per_revolution * gear_ratio / spool_circumfrence
ic(steps_per_mm)
max_rpm = 100.0
max_revs_per_second = max_rpm / 60.0
max_steps_per_second = max_revs_per_second * motor_steps_per_revolution
ic(max_steps_per_second)
max_velocity = max_steps_per_second / steps_per_mm
ic(max_velocity)
max_acceleration = 1.0
max_jerk = 1.0

# Constant Jerk

Reach maximum acceleration

acceleration = max_jerk * t

velocity = 0.5 * max_jerk * t ** 2

t_max_jerk_2 = sqrt(velocity * 2 / max_jerk)

In [ ]:
t_max_jerk = ic(max_acceleration / max_jerk)
t_max_jerk_2 = ic(np.sqrt(max_velocity * 2 / max_jerk))
t_max_jerk = min(t_max_jerk, t_max_jerk_2)

# Constant Acceleration

Reach maximum acceleration

delta_acc = t_max_jerk * max_jerk

In [ ]:
delta_vel = ic(max_velocity) - ic(0.5 * max_jerk * t_max_jerk ** 2)
ic(delta_vel)
t_max_vel = delta_vel / max_acceleration
ic(t_max_vel)

In [ ]:
start = np.array([150.0, 200.0]).reshape(1,2)
end = np.array([0.0, 0.0]).reshape(1,2)

In [ ]:
def s_curve(T, dt):
    coefs = [0.0, 0.0, 0.0, 10/T**3, -15/T**4, 6/T**5]
    t = np.arange(0, T + dt, dt)
    s = coefs[3] * t ** 3 + coefs[4] * t ** 4 + coefs[5] * t ** 5
    s = s.reshape(len(s), 1)
    return t, s

In [ ]:
def motion_curve(start, end, T, dt):
    (t, s) = s_curve(T, dt)
    position = start + s @ (end - start)
    return t, position

In [ ]:
def higher_curves(dt, position):
    velocity = np.diff(position, axis=0) / dt
    acceleration = np.diff(velocity, axis=0) / dt
    jerk = np.diff(acceleration, axis=0) / dt
    return velocity, acceleration, jerk

In [ ]:
def abs_curve(curve):
    return np.linalg.norm(curve, axis=1)

In [ ]:
def compute_scaling(dt, position, max_velocity, max_acceleration, max_jerk):
    vel, acc, jerk = higher_curves(dt, position)
    vel_scaling = ic(ic(abs_curve(vel).max()) / max_velocity)
    acc_scaling = ic((ic(abs_curve(acc).max()) / max_acceleration)**(1/2))
    jerk_scaling = ic((ic(abs_curve(jerk).max()) / max_jerk) ** (1/3))
    return np.max([vel_scaling, acc_scaling, jerk_scaling])

In [ ]:
dt = 0.01
t, position = motion_curve(start, end, 1, dt)
scaling = compute_scaling(dt, position, max_velocity, max_acceleration, max_jerk)
ic(scaling)

In [ ]:
t, position = motion_curve(start, end, scaling, dt)
vel, acc, jerk = higher_curves(dt, position)

In [ ]:
hv.Curve(zip(t, abs_curve(vel)), "t", "vel") + hv.Curve(zip(t, abs_curve(acc)), "t", "acc") + hv.Curve(zip(t, abs_curve(jerk)), "t", "jerk")